In [1]:
from keras import backend as K
import numpy as np

def softmax2d(x):
    # Todo: figure out why we have to save x.shape as a tuple instead of using it directly
    # using it directly causes a keras problem

    old_shape=(-1,int(x.shape[1]),int(x.shape[2]),int(x.shape[3]))
    new_shape=(-1,1,int(x.shape[1])*int(x.shape[2]),old_shape[3])
    nsx = K.reshape(x,new_shape)
    e = K.exp(nsx - K.max(nsx, axis=-2, keepdims=True))
    s = K.sum(e, axis=-2, keepdims=True)
    ns = e / s
    ex = K.reshape(ns, old_shape)
    return ex


# returns x,y coordinates[0-1) of maximum value for each channel
# input should pass through softmax2d before input to this funciton
def softargmax(x):
    xc = K.variable(np.arange(int(x.shape[1]))/(int(x.shape[1])-1))
    x1 = K.variable(np.ones([x.shape[1]]))
    yc = K.variable(np.arange(int(x.shape[2]))/(int(x.shape[2])-1))
    y1 = K.variable(np.ones([x.shape[2]]))
    xx = K.dot(yc, x)
    xx = K.dot(x1, xx)
    xy = K.dot(y1, x)
    xy = K.dot(xc, xy)
    nc=K.stack([xx,xy],axis=-1)
    return nc

Using TensorFlow backend.


In [2]:
from keras.layers import Dense,Input,BatchNormalization,Dropout,Conv2D,AvgPool2D,Flatten,Lambda,Concatenate
from keras.models import Model
import keras
from keras.applications.vgg16 import VGG16

commonkwargs = {"activation": 'relu','padding': 'valid'}
height = 400#160
width = 800 #320
image_size = [width, height]
classes = 2

#three input layers
img_robot_input_TL = Input(shape=(height, width, 3), name='image_robot_input_TL')
img_robot_input_TR = Input(shape=(height, width, 3), name='image_robot_input_TR')
img_robot_input_BL = Input(shape=(height, width, 3), name='image_robot_input_BL')
img_robot_input_BR = Input(shape=(height, width, 3), name='image_robot_input_BR')
img_classifier_input = Input(shape=(height, width, 3), name='image_classifier_input')
pos_input_TL = Input(shape=(2,), name='position_input_TL')
pos_input_TR = Input(shape=(2,), name='position_input_TR')
pos_input_BL = Input(shape=(2,), name='position_input_BL')
pos_input_BR = Input(shape=(2,), name='position_input_BR')
img_input = Input(shape=(height, width, 3), name='image_input')

#conv net and attention layers
x=Conv2D(64, (3,3), strides=(2, 2), **commonkwargs, name='block1_conv1')(img_input)
x=BatchNormalization(name='batch_norm1')(x)
x=Conv2D(64, (3,3), strides=(1, 1), **commonkwargs, name='block1_conv2')(x)
x=BatchNormalization(name='batch_norm2')(x)
x=Conv2D(32, (3,3), strides=(1, 1), **commonkwargs, name='block2_conv1')(x)
x=BatchNormalization(name='batch_norm3')(x)
x=Conv2D(32, (3,3), strides=(1, 1), **commonkwargs, name='block2_conv2')(x)
x=BatchNormalization(name='batch_norm4')(x)
x=Conv2D(16, (3,3), strides=(1, 1), **commonkwargs, name='block3_conv1')(x)
x=BatchNormalization(name='batch_norm5')(x)
x = Lambda(softmax2d, name='image_softmax2d')(x)
x = Lambda(softargmax, name="expected_feature_location")(x)
x = Dropout(.25)(x)
feature_layer = Flatten(name='flattend_feature')(x)
sub_model = Model(img_input, feature_layer, name='feature_model')
sub_model.summary()
sub_model.get_layer('block1_conv1').set_weights(VGG16(weights='imagenet', 
                                                      include_top=False).get_layer('block1_conv1').get_weights())

#classifier layers
classifier_dense1 = Dense(32, activation='relu', name='classifier_dense1')(sub_model(img_classifier_input))
classifier_dense2 = Dense(16, activation='relu', name='classifier_dense2')(classifier_dense1)
classifier_output = Dense(classes, activation='softmax', name='classifier_output')(classifier_dense2)

#robot layers
robot_dense_TL = Dense(16, activation='relu', name='robot_dense_TL')(keras.layers.concatenate(
    [sub_model(img_robot_input_TL), pos_input_TL], name='feature_TL'))
robot_dense_TR = Dense(16, activation='relu', name='robot_dense_TR')(keras.layers.concatenate(
    [sub_model(img_robot_input_TR), pos_input_TR], name='feature_TR'))
robot_dense_BL = Dense(16, activation='relu', name='robot_dense_BL')(keras.layers.concatenate(
    [sub_model(img_robot_input_BL), pos_input_BL], name='feature_BL'))
robot_dense_BR = Dense(16, activation='relu', name='robot_dense_BR')(keras.layers.concatenate(
    [sub_model(img_robot_input_BR), pos_input_BR], name='feature_BR'))
robot_dense1 = Dense(16, activation='relu', name='robot_dense1')(keras.layers.concatenate(
    [robot_dense_TL, robot_dense_TR, robot_dense_BL, robot_dense_BR], name='concatenate_dense'))
robot_dense2 = Dense(16, activation='relu', name='robot_dense2')(robot_dense1)
robot_output = Dense(1, activation='linear', name='robot_output')(robot_dense2)

model = Model([img_robot_input_TL, img_robot_input_TR, img_robot_input_BL, img_robot_input_BR, pos_input_TL, 
               pos_input_TR, pos_input_BL, pos_input_BR, img_classifier_input], [robot_output, classifier_output])
model.compile(optimizer='adam', loss={'robot_output': 'mse', 'classifier_output': 'categorical_crossentropy'}, 
              loss_weights={'robot_output': 1, 'classifier_output': 0}, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 400, 800, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 199, 399, 64)      1792      
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 199, 399, 64)      256       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 197, 397, 64)      36928     
_________________________________________________________________
batch_norm2 (BatchNormalizat (None, 197, 397, 64)      256       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 195, 395, 32)      18464     
_________________________________________________________________
batch_norm3 (BatchNormalizat (None, 195, 395, 32)      128       
__________

In [ ]:
import pydot, graphviz
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [3]:
import os
directory_classifier = "../training_data/classifier/"
directory_robot = "../training_data/4clear/"
classifier_1_data = [directory_classifier + 'peduncle/' + each for each in os.listdir(directory_classifier + 'peduncle')]
classifier_0_data = [directory_classifier + 'cane/' + each for each in os.listdir(directory_classifier + 'cane')]+[directory_classifier + 'grape/' + each for each in os.listdir(directory_classifier + 'grape')]+[directory_classifier + 'leaf/' + each for each in os.listdir(directory_classifier + 'leaf')]
robot_data = [directory_robot + each for each in os.listdir(directory_robot)]
print(len(classifier_0_data), len(classifier_1_data), len(robot_data))
classifier_1_data_training = classifier_1_data[0:50]
classifier_1_data_validation = classifier_1_data[50:]
classifier_0_data_training = classifier_0_data[0:50]
classifier_0_data_validation = classifier_0_data[50:]
robot_data_training = robot_data[0:15]
robot_data_validation = robot_data[15:]
print(len(classifier_1_data_training), len(classifier_1_data_validation), 
      len(classifier_0_data_training), len(classifier_0_data_validation),
      len(robot_data_training), len(robot_data_validation))

662 674 158
50 624 50 612 15 143


In [4]:
import h5py
for i in range(len(robot_data)):
    with h5py.File(robot_data[i], 'r') as f:
        if len(f['reward']) == len(f['image_TL']) == len(f['image_TR']) == len(f['image_BL']) == len(f['image_BR']):
            print(robot_data[i])
        else:
            print('wrong', robot_data[i], len(f['image_TL']), len(f['image_TR']), len(f['image_BL']), len(f['image_BR']))

../training_data/4clear/GOPR0249.hdf5
../training_data/4clear/GOPR0102.hdf5
../training_data/4clear/GOPR0056.hdf5
../training_data/4clear/GOPR0078.hdf5
../training_data/4clear/GOPR0009.hdf5
../training_data/4clear/GOPR0023.hdf5
../training_data/4clear/GOPR0211.hdf5
../training_data/4clear/GOPR0229.hdf5
../training_data/4clear/GOPR0210.hdf5
../training_data/4clear/GOPR0050.hdf5
../training_data/4clear/GOPR0006.hdf5
../training_data/4clear/GOPR0090.hdf5
../training_data/4clear/GOPR0116.hdf5
../training_data/4clear/GOPR0161.hdf5
../training_data/4clear/GOPR0150.hdf5
../training_data/4clear/GOPR0257.hdf5
../training_data/4clear/GOPR0124.hdf5
../training_data/4clear/GOPR0236.hdf5
../training_data/4clear/GOPR0007.hdf5
../training_data/4clear/GOPR0248.hdf5
../training_data/4clear/GOPR0055.hdf5
../training_data/4clear/GOPR0115.hdf5
../training_data/4clear/GOPR0107.hdf5
../training_data/4clear/GOPR0280.hdf5
../training_data/4clear/GOPR0250.hdf5
../training_data/4clear/GOPR0270.hdf5
../training_

In [5]:
import h5py
from PIL import Image
import random


def generate_arrays_from_file(robot_data, classifier_1_data, classifier_0_data, batch_size):
    if batch_size > 50:
        batch_size = 50
    while 1:
        Y_classifier = []
        X_classifier = []
        Y_robot = []
        X_robot_imgTL = []
        X_robot_imgTR = []
        X_robot_imgBL = []
        X_robot_imgBR = []
        X_robot_posTL = []
        X_robot_posTR = []
        X_robot_posBL = []
        X_robot_posBR = []
        with h5py.File(robot_data[random.randrange(len(robot_data))], 'r') as f:
            sampled_frames = random.sample(range(len(f['reward'])), batch_size)
            for i in sampled_frames:
                X_robot_posTL.append([f['cut_position_TL'][0], f['cut_position_TL'][1]])
                X_robot_posTR.append([f['cut_position_TR'][0], f['cut_position_TR'][1]])
                X_robot_posBL.append([f['cut_position_BL'][0], f['cut_position_BL'][1]])
                X_robot_posBR.append([f['cut_position_BR'][0], f['cut_position_BR'][1]])
                #img = Image.fromarray(f['image_TL'][i], 'RGB')
                #img = img.resize(image_size, Image.ANTIALIAS)
                #img.show()
                X_robot_imgTL.append(f['image_TL'][i])
                X_robot_imgTR.append(f['image_TR'][i])
                X_robot_imgBL.append(f['image_BL'][i])
                X_robot_imgBR.append(f['image_BR'][i])
                Y_robot.append(f['reward'][i])
        X_robot_posTL = np.array(X_robot_posTL).astype("float")
        X_robot_posTR = np.array(X_robot_posTR).astype("float")
        X_robot_posBL = np.array(X_robot_posBL).astype("float")
        X_robot_posBR = np.array(X_robot_posBR).astype("float")
        X_robot_imgTL = np.array(X_robot_imgTL).astype("float")
        X_robot_imgTR = np.array(X_robot_imgTR).astype("float")
        X_robot_imgBL = np.array(X_robot_imgBL).astype("float")
        X_robot_imgBR = np.array(X_robot_imgBR).astype("float")
        Y_robot = np.array(Y_robot).astype("float")
        for i in random.sample(range(len(classifier_1_data)), int(batch_size/2)):
            with Image.open(classifier_1_data[i]) as im:
                img = im.resize(image_size, Image.ANTIALIAS)
                X_classifier.append(np.array(img).astype("float"))
                Y_classifier.append([1, 0])
        for i in random.sample(range(len(classifier_0_data)), batch_size - int(batch_size/2)):
            with Image.open(classifier_0_data[i]) as im:
                img = im.resize(image_size, Image.ANTIALIAS)
                X_classifier.append(np.array(img).astype("float"))
                Y_classifier.append([0, 1])
        X_classifier = np.array(X_classifier).astype("float")
        Y_classifier = np.array(Y_classifier).astype("float")
        yield ([X_robot_imgTL/255, X_robot_imgTR/255, X_robot_imgBL/255, X_robot_imgBR/255, X_robot_posTL, X_robot_posTR, 
                X_robot_posBL, X_robot_posBR, X_classifier/255], [Y_robot, Y_classifier])

In [6]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath="./GPLAC_model3.hdf5", verbose=False, save_best_only=True)
K.set_value(model.optimizer.lr, 3e-4)
print(K.get_value(model.optimizer.lr))

0.0003


In [ ]:
batch_size = 8
epochs = 300
history = model.fit_generator(generate_arrays_from_file(robot_data_training, classifier_1_data_training, classifier_0_data_training, batch_size),
                    epochs=epochs, steps_per_epoch=5, validation_steps=1,
                    validation_data=generate_arrays_from_file(robot_data_validation, classifier_1_data_validation, classifier_0_data_validation, batch_size),
                    callbacks=[checkpointer])

Epoch 1/300
5/5 [==============================] - 34s - loss: 934.8125 - robot_output_loss: 934.8125 - classifier_output_loss: 0.7029 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.5250 - val_loss: 1426.6447 - val_robot_output_loss: 1426.6447 - val_classifier_output_loss: 0.6923 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.5000
Epoch 2/300
5/5 [==============================] - 20s - loss: 720.3634 - robot_output_loss: 720.3634 - classifier_output_loss: 0.7010 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.4750 - val_loss: 949.9282 - val_robot_output_loss: 949.9282 - val_classifier_output_loss: 0.6931 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.5000
Epoch 3/300
5/5 [==============================] - 20s - loss: 776.0436 - robot_output_loss: 776.0436 - classifier_output_loss: 0.7026 - robot_output_acc: 0.0250 - classifier_output_acc: 0.5750 - val_loss: 1210.2406 - val_robot_output_loss: 1210.2406 - val_classifier_output_l

5/5 [==============================] - 19s - loss: 752.2398 - robot_output_loss: 752.2398 - classifier_output_loss: 0.6918 - robot_output_acc: 0.0250 - classifier_output_acc: 0.4750 - val_loss: 1379.7605 - val_robot_output_loss: 1379.7605 - val_classifier_output_loss: 0.6903 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.5000
Epoch 24/300
5/5 [==============================] - 20s - loss: 823.3357 - robot_output_loss: 823.3357 - classifier_output_loss: 0.7026 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.5250 - val_loss: 735.2352 - val_robot_output_loss: 735.2352 - val_classifier_output_loss: 0.6977 - val_robot_output_acc: 0.1250 - val_classifier_output_acc: 0.3750
Epoch 25/300
5/5 [==============================] - 20s - loss: 973.8575 - robot_output_loss: 973.8575 - classifier_output_loss: 0.7064 - robot_output_acc: 0.0250 - classifier_output_acc: 0.4250 - val_loss: 733.5554 - val_robot_output_loss: 733.5554 - val_classifier_output_loss: 0.7037 - val_ro

5/5 [==============================] - 19s - loss: 949.0892 - robot_output_loss: 949.0892 - classifier_output_loss: 0.7224 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.5500 - val_loss: 1391.1162 - val_robot_output_loss: 1391.1162 - val_classifier_output_loss: 0.6916 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.6250
Epoch 46/300
5/5 [==============================] - 19s - loss: 1065.4420 - robot_output_loss: 1065.4420 - classifier_output_loss: 0.7183 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.3750 - val_loss: 1050.3579 - val_robot_output_loss: 1050.3579 - val_classifier_output_loss: 0.6886 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.5000
Epoch 47/300
5/5 [==============================] - 19s - loss: 735.1604 - robot_output_loss: 735.1604 - classifier_output_loss: 0.7242 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.4750 - val_loss: 516.3797 - val_robot_output_loss: 516.3797 - val_classifier_output_loss:

5/5 [==============================] - 19s - loss: 1111.7132 - robot_output_loss: 1111.7132 - classifier_output_loss: 0.7113 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.4500 - val_loss: 768.1337 - val_robot_output_loss: 768.1337 - val_classifier_output_loss: 0.6874 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.7500
Epoch 68/300
5/5 [==============================] - 19s - loss: 723.7430 - robot_output_loss: 723.7430 - classifier_output_loss: 0.7224 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.3750 - val_loss: 449.3154 - val_robot_output_loss: 449.3154 - val_classifier_output_loss: 0.6974 - val_robot_output_acc: 0.1250 - val_classifier_output_acc: 0.6250
Epoch 69/300
5/5 [==============================] - 19s - loss: 779.5619 - robot_output_loss: 779.5619 - classifier_output_loss: 0.7092 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.4750 - val_loss: 874.0362 - val_robot_output_loss: 874.0362 - val_classifier_output_loss: 0.6895 

5/5 [==============================] - 19s - loss: 790.2890 - robot_output_loss: 790.2890 - classifier_output_loss: 0.7939 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.2250 - val_loss: 598.0671 - val_robot_output_loss: 598.0671 - val_classifier_output_loss: 0.7098 - val_robot_output_acc: 0.1250 - val_classifier_output_acc: 0.5000
Epoch 90/300
5/5 [==============================] - 18s - loss: 756.0185 - robot_output_loss: 756.0185 - classifier_output_loss: 0.7026 - robot_output_acc: 0.0250 - classifier_output_acc: 0.4500 - val_loss: 845.4377 - val_robot_output_loss: 845.4377 - val_classifier_output_loss: 0.6778 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.6250
Epoch 91/300
5/5 [==============================] - 19s - loss: 842.0690 - robot_output_loss: 842.0690 - classifier_output_loss: 0.7017 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.4750 - val_loss: 928.0073 - val_robot_output_loss: 928.0073 - val_classifier_output_loss: 0.6888 - val_

5/5 [==============================] - 20s - loss: 665.2853 - robot_output_loss: 665.2853 - classifier_output_loss: 0.7090 - robot_output_acc: 0.0250 - classifier_output_acc: 0.5000 - val_loss: 1024.8793 - val_robot_output_loss: 1024.8793 - val_classifier_output_loss: 0.6593 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.8750
Epoch 112/300
5/5 [==============================] - 19s - loss: 1005.0757 - robot_output_loss: 1005.0757 - classifier_output_loss: 0.7110 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.4250 - val_loss: 755.6921 - val_robot_output_loss: 755.6921 - val_classifier_output_loss: 0.6618 - val_robot_output_acc: 0.0000e+00 - val_classifier_output_acc: 0.6250
Epoch 113/300
5/5 [==============================] - 19s - loss: 1018.6903 - robot_output_loss: 1018.6903 - classifier_output_loss: 0.7103 - robot_output_acc: 0.0000e+00 - classifier_output_acc: 0.4500 - val_loss: 1291.5037 - val_robot_output_loss: 1291.5037 - val_classifier_output_loss: